<a href="https://colab.research.google.com/github/M4700F/BERT/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install pandas
!pip install optuna
!pip install torch
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 21.2 MB/s eta 0:00:00

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Copyphishing.csv to Copyphishing.csv


In [ ]:
import pandas as pd

dataset = pd.read_csv("Copyphishing.csv")

cleaned_dataset = dataset[['Feature', 'Boolean']].dropna()
cleaned_dataset = cleaned_dataset.rename(columns={'Feature': 'text', 'Boolean': 'label'})

cleaned_dataset['label'] = cleaned_dataset['label'].astype(int)

print(cleaned_dataset.head())

                                                text  label
0  Dear eBay User ,\n\nAfter fraud complaints fro...      1
1  Dear valued customer, Our records indicate tha...      1
2  Dear Key Bank customer.  Please read this mess...      1
3  Dear Key Bank customer.  Please read this mess...      1
4  LEGAL NOTICE The following message is an email...      1


In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset

# loading BERT Tokenizer...
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

dataset = Dataset.from_pandas(cleaned_dataset)

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# applying tokenization on each batch
tokenized_dataset = dataset.map(tokenize_function, batched=True)

print("Tokenized Data : \n")
print(tokenized_dataset[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/8194 [00:00<?, ? examples/s]

Tokenized Data : 

{'text': "Dear eBay User ,\\n\\nAfter fraud complaints from the eBay members, the eBay Inc. had\\ndeveloped a security program against the fraudulend attempts of accounts\\nthefts. For that we have to securise all the members informations by\\nupdating and checking the registrated informations. Please update  your\\ninformation by completing the form from the forwarded link so we can check\\nyour account validity and your identity\\nand login to eBay in order to update your informations.\\nThis process will take 5 days, period when you will not be able to\\nacces your eBay account. After this period you will receive instructions to\\nenter and securise your eBay account.Please click the link below and sign in into your account: http://signin.ebay.com/aw-cgi/eBayISAPI.dll?SignIn&ssPageName=h:h:sin:US -- As outlined in our User Agreement, eBay will periodically send you information about site changes and enhancements. Visit our Privacy Policy and User Agreement if you 

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import torch

# training -> 80%, validation -> 20%
train_dataset, val_dataset = tokenized_dataset.train_test_split(test_size=0.2).values()

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    report_to="none"
)

# initializing ...
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-5ab7b8785928>:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.000400,0.006662
2,0.027800,0.006381
3,0.000100,0.006841


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-5ab7b8785928>", line 33, in <cell line: 33>
    trainer.train()
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2164, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2577, in _inner_training_loop
    self.optimizer.step()
  File "/usr/local/lib/python3.10/dist-packages/accelerate/optimizer.py", line 178, in step
    self.optimizer.step(closure)
  File "/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py", line 137, in wrapper
    return func.__get__(opt, opt.__class__)(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/optim/optimizer.py", line 487, in wrapper
    out = func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-pac

TypeError: object of type 'NoneType' has no len()

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    predictions = predictions.cpu().numpy()
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}



trainer.compute_metrics=compute_metrics

evaluation_results = trainer.evaluate()
print("Evaluation Metrics:", evaluation_results)

In [ ]:
from torch.nn import CrossEntropyLoss

class_weights = torch.tensor([1.0, 2.0])
loss_fn = CrossEntropyLoss(weight=class_weights)
model.config.problem_type = "binary_classification"


In [ ]:
import optuna # optuna is a open-source hyperparameter optimization framework
from transformers import Trainer, TrainingArguments

def model_training(trial):

    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    num_epochs = trial.suggest_int("num_epochs", 3, 6)

    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        num_train_epochs=num_epochs,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
    )

    trainer.train()
    evaluation_results = trainer.evaluate()
    return evaluation_results["eval_f1"]

# Running Optuna study ...
study = optuna.create_study(direction="maximize")
study.optimize(model_training, n_trials=10)

print("Best Hyperparameters:", study.best_params)


In [ ]:
model.save_pretrained("./phishing_model")
tokenizer.save_pretrained("./phishing_model")


In [ ]:
# for loading the model and tokenizer later
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("./phishing_model")
tokenizer = AutoTokenizer.from_pretrained("./phishing_model")


In [ ]:
def predict_email(email_text):

    inputs = tokenizer(email_text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)

    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()  # 0: Non-phishing, 1: Phishing

    return "Phishing" if predicted_class == 1 else "Non-phishing"

email = "You have won $1,000,000. Click here to claim your prize!"
print(predict_email(email))
